In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
import plotly.figure_factory as ff
import numpy as np

## Lectura y limpieza datos

https://www.kaggle.com/datasets/utkarshx27/movies-dataset?resource=download

#### Lectura

In [2]:
df = pd.read_csv('../data/raw/movie_dataset.csv', index_col='index')
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
index,,,,,,,,,,,,,,,,,,,,,
0,237000000,Action Adventure Fantasy Science_Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,260000000,Action Adventure Science_Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
df.columns
# Target = revenue

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

#### Eliminar columnas no usadas

In [4]:
# Remove unused columns:
df.drop(columns=['homepage', 'id', 'keywords', 'overview', 'status', 'tagline', 'original_title', 'crew', 'cast', 'spoken_languages'], inplace=True)


#### Separar géneros en columnas

In [5]:
# separate genres into dummies
df_genres = df['genres'].str.get_dummies(sep=' ')

df_final = pd.concat([df, df_genres], axis=1)
df_final

,budget,genres,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,title,...,History,Horror,Music,Mystery,Romance,Science_Fiction,TV_Movie,Thriller,War,Western
index,,,,,,,,,,,,,,,,,,,,,
0,237000000,Action Adventure Fantasy Science_Fiction,en,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,Avatar,...,0,0,0,0,0,1,0,0,0,0
1,300000000,Adventure Fantasy Action,en,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,Pirates of the Caribbean: At World's End,...,0,0,0,0,0,0,0,0,0,0
2,245000000,Action Adventure Crime,en,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,Spectre,...,0,0,0,0,0,0,0,0,0,0
3,250000000,Action Crime Drama Thriller,en,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,The Dark Knight Rises,...,0,0,0,0,0,0,0,1,0,0
4,260000000,Action Adventure Science_Fiction,en,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,John Carter,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,Action Crime Thriller,es,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,El Mariachi,...,0,0,0,0,0,0,0,1,0,0
4799,9000,Comedy Romance,en,0.642552,[],[],2011-12-26,0,85.0,Newlyweds,...,0,0,0,0,1,0,0,0,0,0
4800,0,Comedy Drama Romance TV_Movie,en,1.444476,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-10-13,0,120.0,"Signed, Sealed, Delivered",...,0,0,0,0,1,0,1,0,0,0


#### Extraer Production Company

In [62]:
# Extraer Production Company
df['production_companies'][0]

['Other_company',
 'Twentieth Century Fox Film Corporation',
 'Other_company',
 'Other_company']

In [6]:
df['production_companies'] = df['production_companies'].apply(lambda x: [d['name'] for d in ast.literal_eval(x)]) 

# Contar la frecuencia de cada compañía en todas las filas
all_companies = [company for companies_list in df['production_companies'] for company in companies_list]
top_companies = [company for company, count in Counter(all_companies).most_common(10)]

# Crear una lista de compañías que estén en el top 5 o "Otros"
df['production_companies'] = df['production_companies'].apply(lambda x: [company if company in top_companies else "Other_company" for company in x])

# Aplicar One-Hot Encoding a las compañías seleccionadas (incluyendo "Otros")
mlb = MultiLabelBinarizer()
df_companies = pd.DataFrame(mlb.fit_transform(df['production_companies']), columns=mlb.classes_, index=df.index)
df_companies.head()

,Columbia Pictures,Metro-Goldwyn-Mayer (MGM),New Line Cinema,Other_company,Paramount Pictures,Relativity Media,Touchstone Pictures,Twentieth Century Fox Film Corporation,Universal Pictures,Walt Disney Pictures,Warner Bros.
index,,,,,,,,,,,
0,0,0,0,1,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,1,0
2,1,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1,0


In [163]:
df_final = pd.concat([df_final, df_companies], axis=1)

#### Extraer Production Country

In [164]:
df['production_countries'] = df['production_countries'].apply(lambda x: [d['name'] for d in ast.literal_eval(x)])


# Contar la frecuencia de cada compañía en todas las filas
all_countries = [country for countries_list in df['production_countries'] for country in countries_list]
top_countries = [company for company, count in Counter(all_countries).most_common(5)]

# Crear una lista de compañías que estén en el top 5 o "Otros"
df['production_countries'] = df['production_countries'].apply(lambda x: [company if company in top_countries else "Other_country" for company in x])

# Aplicar One-Hot Encoding a las compañías seleccionadas (incluyendo "Otros")
mlb = MultiLabelBinarizer()
df_countries = pd.DataFrame(mlb.fit_transform(df['production_countries']), columns=mlb.classes_, index=df.index)
df_countries.head()



,Canada,France,Germany,Other_country,United Kingdom,United States of America
index,,,,,,
0,0,0,0,0,1,1
1,0,0,0,0,0,1
2,0,0,0,0,1,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [165]:
df_final = pd.concat([df_final, df_countries], axis=1)

In [166]:
df_final['release_date'] = pd.to_datetime(df_final['release_date'])

In [167]:
df_final.columns

Index(['budget', 'genres', 'original_language', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'title', 'vote_average', 'vote_count', 'director',
       'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Fiction', 'Foreign', 'History', 'Horror',
       'Movie', 'Music', 'Mystery', 'Romance', 'Science', 'TV', 'Thriller',
       'War', 'Western', 'Columbia Pictures', 'Other_company',
       'Paramount Pictures', 'Twentieth Century Fox Film Corporation',
       'Universal Pictures', 'Warner Bros.', 'Canada', 'France', 'Germany',
       'Other_country', 'United Kingdom', 'United States of America'],
      dtype='object')

In [168]:
df_final = df_final.dropna()

In [169]:
df_plots = df_final.copy()
df_model = df_final.drop(columns=["genres", "production_companies", "production_countries", "title", "director"])

#### Outliers?

In [170]:
# Calcular los cuartiles y el IQR
Q1 = df['revenue'].quantile(0.25)
Q3 = df['revenue'].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites inferior y superior para los outliers
lower_limit = Q1 - 10 * IQR
upper_limit = Q3 + 10 * IQR

# Filtrar los outliers
outliers = df[(df['revenue'] < lower_limit) | (df['revenue'] > upper_limit)]

# Mostrar los outliers
print("Outliers detectados usando IQR:")
outliers

"""
No son outliers reales, son valores extremos válidos, valorar si afectan demasiado al modelo cuando se llegue a esa parte
"""

Outliers detectados usando IQR:


'\nNo son outliers reales, son valores extremos válidos, valorar si afectan demasiado al modelo cuando se llegue a esa parte\n'

## Gráficos

#### Top Géneros

In [171]:
# Dividir los géneros en cada fila y contar la frecuencia de cada uno
all_genres = [genre for genres_list in df_plots['genres'].str.split() for genre in genres_list]
genre_counts = Counter(all_genres)

# Seleccionar los 10 géneros más comunes
top_10_genres = genre_counts.most_common(10)
genres, counts = zip(*top_10_genres)

# # Crear el gráfico de barras con Plotly
# fig = px.bar(x=genres, y=counts, title="Top 10 Géneros de Películas", labels={'x': 'Género', 'y': 'Frecuencia'})
# fig.show()

# Crear el gráfico de barras con plotly.graph_objects
fig = go.Figure()

fig.add_trace(go.Bar(
    x=list(genres),
    y=list(counts),
    marker=dict(color='skyblue'),
))

# Personalizar el gráfico
fig.update_layout(
    title="Top 10 Géneros de Películas",
    xaxis_title="Género",
    yaxis_title="Frecuencia",
    template="plotly_white",
    width=800
)

# Mostrar el gráfico
fig.show()

In [124]:
df = df_plots.copy()

df["genres_list"] = df["genres"].str.split()

# Desplegar las filas para cada género con su respectivo revenue
genre_avg_revenue = (
    df.explode("genres_list")[["genres_list", "revenue"]].groupby("genres_list")["revenue"].mean().reset_index()
).sort_values(by="revenue", ascending=True)
top_10_genres = genre_avg_revenue.tail(10)

# Paso 6: Separar en dos listas para poder usarlas en un gráfico o análisis posterior
genres, avg_revenue = top_10_genres["genres_list"], top_10_genres["revenue"]

# Crear el gráfico de barras con plotly.graph_objects
fig = go.Figure()

fig.add_trace(go.Bar(y=genres, x=avg_revenue, marker=dict(color="skyblue"), orientation="h"))

# Personalizar el gráfico
fig.update_layout(
    title="Top 10 Géneros de Películas",
    yaxis_title="Género",
    xaxis_title="Recaudación Media",
    width=800,
    plot_bgcolor="#F3F6FF",
    paper_bgcolor="#F3F6FF",
    font=dict(size=8),
    margin=dict(l=0, r=0, b=0, t=25, pad=4),
)

In [148]:
top_10_genres

,genres_list,revenue
2,Crime,880674609
3,Crime,1084939099
3,Drama,1084939099
3,Thriller,1084939099
10,Thriller,586090727
...,...,...
1225,Drama,416498
1226,Thriller,600896
1227,Drama,10000
1229,Drama,424760


In [ ]:
# Paso 1: Desplegar los géneros en una nueva columna (según cómo estén en tu DataFrame)
df["genres_list"] = df["genres"].str.split()

# Paso 2: Desplegar las filas para cada género con su respectivo revenue
genre_revenue_df = df.explode("genres_list")[["genres_list", "revenue"]]

# Paso 3: Agrupar por género y calcular la media de revenue
genre_avg_revenue = genre_revenue_df.groupby("genres_list")["revenue"].mean().reset_index()

# Paso 4: Ordenar por la media de revenue de mayor a menor
top_10_genres_revenues = genre_avg_revenue.sort_values(by="revenue", ascending=True).tail(5)

# Paso 3: Filtrar el DataFrame original para contener solo estos top 5 directores
top_10_genres_names = top_10_genres_revenues["genres_list"]
top_10_genres = genre_revenue_df[genre_revenue_df["genres_list"].isin(top_10_genres_names)]

# Paso 6: Crear el box plot usando `go`
fig = go.Figure()

# Añadir un box plot para cada género
for genre in top_10_genres_names:
    genre_data = top_10_genres[top_10_genres["genres_list"] == genre]["revenue"]
    genre_mean = genre_data.mean()  # Calcular la media para este género

    # Box plot para el género
    fig.add_trace(
        go.Box(
            x=genre_data,
            name=genre,
            boxmean=True,  # Deshabilitar la media automática
            marker_color="#488BFF",
            boxpoints="outliers",  # Mostrar solo los outliers como puntos
        )
    )

# Configurar el layout del gráfico
fig.update_layout(
    title="Distribución de Recaudación por Género (Top 10)",
    yaxis_title="Género",
    xaxis_title="Recaudación",
    template="plotly_white",
    xaxis=dict(tickangle=-45),  # Rotar las etiquetas de los géneros
    showlegend=False  # Ocultar leyendas para los puntos medios
)

# Mostrar el gráfico
fig.show()


#### Histograma Revenue

In [172]:
data=[go.Histogram(x = df_plots['revenue'], 
            marker = dict(color='skyblue'),
            showlegend=False)]

layout =go.Layout(
            title = 'Histograma Revenue',
            titlefont={'size': 15},
            font=dict(size=8),
            width=800,
            # margin=dict(
            #     l=25,
            #     r=0,
            #     b=0,
            #     t=25,
            #     pad=4
            # )
        )
go.Figure(data=data, layout=layout)

#### Evolución revenue por año

In [ ]:
"""
No tiene sentido porque una película puede tener muchas productoras
"""

# # Crear una columna con el año de la fecha de lanzamiento
# df_plots['release_year'] = df_plots['release_date'].dt.year

# # Calcular el promedio de revenue por año y por productora
# df_aux = df_plots.groupby(['release_year', 'Warner Bros.', 'Universal Pictures', 'Paramount Pictures', 
#                            'Twentieth Century Fox Film Corporation', 'Columbia Pictures', 'Other_company'])['revenue'].mean().reset_index()

# # Crear la figura
# fig = go.Figure()

# # Añadir trazas para cada productora (por cada columna de productoras)
# fig.add_trace(go.Scatter(
#     x=df_aux['release_year'], 
#     y=df_aux['Warner Bros.'],
#     hoverinfo='x+y',
#     mode='lines',
#     line=dict(width=0.5, color='purple'),
#     stackgroup='one',
#     name='Warner Bros.'
# ))

# fig.add_trace(go.Scatter(
#     x=df_aux['release_year'], 
#     y=df_aux['Universal Pictures'],
#     hoverinfo='x+y',
#     mode='lines',
#     line=dict(width=0.5, color='blue'),
#     stackgroup='one',
#     name='Universal Pictures'
# ))

# fig.add_trace(go.Scatter(
#     x=df_aux['release_year'], 
#     y=df_aux['Paramount Pictures'],
#     hoverinfo='x+y',
#     mode='lines',
#     line=dict(width=0.5, color='green'),
#     stackgroup='one',
#     name='Paramount Pictures'
# ))

# fig.add_trace(go.Scatter(
#     x=df_aux['release_year'], 
#     y=df_aux['Twentieth Century Fox Film Corporation'],
#     hoverinfo='x+y',
#     mode='lines',
#     line=dict(width=0.5, color='orange'),
#     stackgroup='one',
#     name='Twentieth Century Fox'
# ))

# fig.add_trace(go.Scatter(
#     x=df_aux['release_year'], 
#     y=df_aux['Columbia Pictures'],
#     hoverinfo='x+y',
#     mode='lines',
#     line=dict(width=0.5, color='red'),
#     stackgroup='one',
#     name='Columbia Pictures'
# ))

# fig.add_trace(go.Scatter(
#     x=df_aux['release_year'], 
#     y=df_aux['Other_company'],
#     hoverinfo='x+y',
#     mode='lines',
#     line=dict(width=0.5, color='gray'),
#     stackgroup='one',
#     name='Otros'
# ))

# # Actualizar el layout para personalizar el gráfico
# fig.update_layout(
#     title='Evolución Temporal de Revenue por Productora',
#     xaxis_title='Año',
#     yaxis_title='Revenue Promedio (Millones)',
#     legend=dict(
#         bgcolor='white',
#         yanchor="top",
#         y=0.99,
#         xanchor="left",
#         x=0.01
#     ),
#     font=dict(size=8),
#     # margin=dict(l=0, r=0, b=0, t=25, pad=4)
# )

# # Mostrar el gráfico
# fig.show()

In [188]:
df_aux = df_plots.groupby(df_plots["release_date"].dt.year)['revenue'].mean().reset_index()[:-1]


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_aux['release_date'], y=df_aux['revenue'],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='steelblue'),
    stackgroup='one', # define stack group
    name = 'Revenue'
))

fig.update_layout(
    title = 'Evolución Temporal',
    xaxis_title = 'Año',
    yaxis_title = 'Ventas (Millones)',
    legend=dict(
        bgcolor = 'white',
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01),
    font=dict(size=8),
    # margin=dict(
    #     l=0,
    #     r=0,
    #     b=0,
    #     t=25,
    #     pad=4
    # )
)

In [ ]:
df_aux = df_plots[df_plots['Action']==1].groupby(df_plots["release_date"].dt.year)['revenue'].mean().reset_index()[:-1]

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = df_aux["release_date"],
        y = df_aux["revenue"],
        mode="lines",
        marker_color='steelblue'
    )
)
fig.update_layout(
    title = f"Revenue", 
    xaxis_title = "Año", 
    yaxis_title = "Revenue", 
    bargap = 0.1,
)

# Crear en la app una página para filtrar y ver la evolución para un género concreto, o cualquier otra variable

#### Matriz de correlación

https://plotly.com/python/builtin-colorscales/

In [197]:
df_num = df.select_dtypes(include='number')
corr = df_num.corr()
z = round(corr,4).values
z = np.flip(z,axis=0)

x = [x for x in df_num.columns]
y = [x for x in df_num.columns]
y.reverse()

z_text = [[str(y) for y in x] for x in z]

heatmap = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='teal')

heatmap.update_layout(
        title = 'Matriz de correlación',
        title_y = 0.95,
        titlefont={'size': 15},
        font=dict(size=8),
        # margin=dict(
        #     l=0,
        #     r=0,
        #     b=0,
        #     t=100,
        #     pad=4
        # )
)

In [ ]:
df_num = df_plots.select_dtypes(include='number')
corr = df_num.corr()
z = round(corr,4).values
z = np.flip(z,axis=0)

x = [x for x in df_num.columns]
y = [x for x in df_num.columns]
y.reverse()

z_text = [[str(y) for y in x] for x in z]

heatmap = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='teal')

heatmap.update_layout(
        title = 'Matriz de correlación',
        title_y = 0.95,
        titlefont={'size': 15},
        font=dict(size=8),
        # margin=dict(
        #     l=0,
        #     r=0,
        #     b=0,
        #     t=100,
        #     pad=4
        # )
)

"""
Science Fiction y TV Movie eran lo mismo, no separar al crear las dummy de géneros
"""

In [217]:
select_year_value = 2010
df_aux_scores = df_plots[df_plots['release_year']==select_year_value].groupby("genres")[['vote_average','vote_count']].mean().reset_index()
df_aux_sales = df_plots[df_plots['release_year']==select_year_value].groupby("genres")['revenue'].mean().reset_index()
df_aux2 = pd.concat([df_aux_scores,df_aux_sales['revenue']],axis=1)

fig = px.scatter(df_aux2, x="vote_average", y="vote_count",
            size="revenue", color="genres",
                hover_name="genres", log_x=True, size_max=60,
                color_discrete_sequence=px.colors.qualitative.Pastel)

fig.update_layout(
        title = "Puntuaciones de usuarios y críticos según el Género y Ventas Globales para el año " + str(select_year_value), 
        xaxis_title = "vote_average", 
        yaxis_title = "vote_count", 
        bargap = 0.1,
        titlefont={'size': 15},
    )


In [216]:
df_plots_expanded = df_plots.copy()

# Dividir la columna 'genres' en múltiples géneros y 'explotar' las filas
df_plots_expanded['genres'] = df_plots_expanded['genres'].str.split()  # separa por espacio
df_plots_expanded = df_plots_expanded.explode('genres')  # crea una fila para cada género

# Paso 2: Filtrar para el año seleccionado
select_year_value = 2010
df_filtered = df_plots_expanded[df_plots_expanded['release_year'] == select_year_value]

# Paso 3: Agrupar por género y calcular la media de 'vote_average' y 'vote_count' y la suma de 'revenue'
df_aux_scores = df_filtered.groupby('genres')[['vote_average', 'vote_count']].mean().reset_index()
df_aux_sales = df_filtered.groupby('genres')['revenue'].mean().reset_index()

# Unir los DataFrames de scores y ventas
df_aux2 = pd.merge(df_aux_scores, df_aux_sales[['genres', 'revenue']], on='genres')

# Paso 4: Crear el gráfico
fig = px.scatter(df_aux2, x="vote_average", y="vote_count",
                 size="revenue", color="genres",
                 hover_name="genres", log_x=True, size_max=60,
                 color_discrete_sequence=px.colors.qualitative.Pastel)

# Actualizar el diseño
fig.update_layout(
    title=f"Puntuaciones de usuarios y críticos según el Género y Ventas Globales para el año {select_year_value}", 
    xaxis_title="vote_average", 
    yaxis_title="vote_count", 
    bargap=0.1,
    titlefont={'size': 15},
)

fig.show()

### directores

In [ ]:
# Paso 4: Agrupar por director y calcular la media de revenue
director_avg_revenue = df.groupby("director")["revenue"].mean().reset_index()
top10directors = director_avg_revenue.sort_values(by="revenue").tail(5)

# Paso 5: Crear el bar chart para los directores con más de una película
fig = px.bar(
    top10directors,
    x="director",   # Eje X: Director
    y="revenue",    # Eje Y: Revenue
    title="Media de Recaudación por Director",
    labels={"director": "Director", "revenue": "Media de Recaudación"},
    color="director",  # Colorear las barras por director
    text="revenue",    # Añadir el valor de revenue encima de cada barra
)

# Mostrar el gráfico
fig.show()

In [132]:
df = df_plots.copy()
# Paso 1: Agrupar por director y calcular la media de revenue
director_avg_revenue = df.groupby("director")["revenue"].mean().reset_index()

# Paso 2: Ordenar por media de revenue y seleccionar los top 5 directores
top_5_directors = director_avg_revenue.sort_values(by="revenue", ascending=False).head(10)

# Paso 3: Filtrar el DataFrame original para contener solo estos top 5 directores
top_5_directors_names = top_5_directors["director"]
df_top_5_directors = df[df["director"].isin(top_5_directors_names)]

# Paso 4: Crear el box plot
fig = px.box(
    df_top_5_directors,
    x="director",    # Eje X: Director
    y="revenue",     # Eje Y: Revenue
    title="Box Plot del Revenue por Director (Top 5 con Mayor Media de Recaudación)",
    labels={"director": "Director", "revenue": "Revenue (en millones)"},
    color="director", # Colorear cada box por director
)

# Mostrar el gráfico
fig.show()

## Datos GDP

Real:
https://www.measuringworth.com/datasets/usgdp/result.php

Forecast:
https://www.statista.com/statistics/216985/forecast-of-us-gross-domestic-product/

In [25]:
gdp_data = pd.read_csv("../data/raw/USGDP_1900-2024.csv", thousands=",")
gdp_data.head()

,Year,Nominal GDP (million of Dollars),Real GDP (millions of 2017 dollars),GDP Deflator (index 2017=100),Population,Nominal GDP per capita (current dollars),Real GDP per capita (year 2017 dollars)
0,1900,21197,521778,4.06,76094000,278.56,6857.02
1,1901,22931,549046,4.18,77584000,295.56,7076.79
2,1902,24754,576747,4.29,79163000,312.69,7285.57
3,1903,26647,592997,4.49,80632000,330.48,7354.36
4,1904,26360,571371,4.61,82166000,320.82,6953.86


In [26]:
real_gdp = gdp_data[['Year','Real GDP (millions of 2017 dollars)']]
real_gdp.columns = ['Year', 'Real_GDP']
real_gdp

,Year,Real_GDP
0,1900,521778
1,1901,549046
2,1902,576747
3,1903,592997
4,1904,571371
...,...,...
119,2019,20715670
120,2020,20267590
121,2021,21494800
122,2022,22034830


In [27]:
px.line(real_gdp, x='Year',y='Real_GDP')

In [29]:
df_plots = pd.read_csv("../data/processed/df_plots.csv")

In [36]:
df_merged = df_plots.merge(real_gdp, left_on='year', right_on='Year', how='left')
df_merged.drop(columns='Year', inplace=True)

In [41]:
df_aux = df_merged.groupby(df_merged["year"])[['revenue','Real_GDP']].mean().reset_index()[:-1]


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_aux['year'], y=df_aux['revenue'],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='steelblue'),
    name = 'Revenue'
))
fig.add_trace(go.Scatter(
    x=df_aux['year'], y=df_aux['Real_GDP'],
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='firebrick'),
    name = 'Revenue real'
))

fig.update_layout(
    title = 'Evolución Temporal',
    xaxis_title = 'Año',
    yaxis_title = 'Ventas (Millones)',
    legend=dict(
        bgcolor = 'white',
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01),
    font=dict(size=8),
    # margin=dict(
    #     l=0,
    #     r=0,
    #     b=0,
    #     t=25,
    #     pad=4
    # )
)

In [46]:
df_plots.loc[df_plots['year']==1942, ["title","revenue"]]

,title,revenue
4312,Casablanca,10462500
4418,Bambi,267447150


In [52]:
df_plots = df_plots[(df_plots['budget']!=0)&(df_plots['revenue']!=0)].reset_index(drop=True)

In [181]:
df_plots = pd.read_csv("../data/processed/df_plots.csv")

In [183]:
# prod_companies = ['Columbia Pictures',
#        'Metro-Goldwyn-Mayer (MGM)', 'New Line Cinema',
#        'Paramount Pictures', 'Relativity Media', 'Touchstone Pictures',
#        'Twentieth Century Fox Film Corporation', 'Universal Pictures',
#        'Walt Disney Pictures', 'Warner Bros.']
prod_companies = [
        "Columbia Pictures",
        "Dune Entertainment",
        "New Line Cinema",
        "Paramount Pictures",
        "Relativity Media",
        "Village Roadshow Pictures",
        "Twentieth Century Fox Film Corporation",
        "Universal Pictures",
        "Walt Disney Pictures",
        "Warner Bros.",
    ]
use_cols = prod_companies + ['revenue']
df_prod_companies = df_plots[use_cols]


In [184]:
df_long = df_prod_companies.melt(
    id_vars=['revenue'], 
    value_vars=prod_companies,
    var_name='production_company', 
    value_name='present'
)

# Filtrar filas donde 'present' es 1
df_long = df_long[df_long['present'] == 1]

# Step 2: Add a new column for the grouped category
df_long['group'] = df_long['production_company'].replace({
    'Twentieth Century Fox Film Corporation': 'Disney Group',
    'Walt Disney Pictures': 'Disney Group'
})

# Step 3: Combine group and individual data
# Create a combined DataFrame that includes both individual and group categories
df_combined = pd.concat([
    df_long,  # Individual companies
    df_long[df_long['group'] == 'Disney Group'].assign(production_company='Disney Group')  # Group category
])


# Step 4: Group by company and calculate metrics
df_grouped = df_combined.groupby('production_company').agg(
    count=('revenue', 'size'),
    avg_revenue=('revenue', 'mean')
).reset_index()

# Step 5: Create the scatter plot
fig = px.scatter(
    df_grouped,
    x='count',
    y='avg_revenue',
    color='production_company',
    text='production_company',
    labels={
        'count': 'Number of Movies',
        'avg_revenue': 'Average Revenue'
    },
    title='Production Companies: Number of Movies vs Average Revenue',
    template='plotly_white'
)

# Add labels
fig.update_traces(textposition='top center')

# Show plot
fig.show()

In [106]:
df_long = df_prod_companies.melt(
    id_vars=['revenue'], 
    value_vars=prod_companies,
    var_name='production_company', 
    value_name='present'
)

# Filtrar filas donde 'present' es 1
df_long = df_long[df_long['present'] == 1]

# Step 2: Add a new column for the grouped category
df_long['group'] = df_long['production_company'].replace({
    'Twentieth Century Fox Film Corporation': 'Disney Group',
    'Walt Disney Pictures': 'Disney Group'
})

# Step 3: Combine group and individual data
# Create a combined DataFrame that includes both individual and group categories
df_combined = pd.concat([
    df_long,  # Individual companies
    df_long[df_long['group'] == 'Disney Group'].assign(production_company='Disney Group')  # Group category
])


# Step 4: Group by company and calculate metrics
df_grouped = df_combined.groupby('production_company').agg(
    count=('revenue', 'size'),
    avg_revenue=('revenue', 'mean')
).reset_index()

# Step 5: Create the scatter plot
fig = px.scatter(
    df_grouped,
    x='count',
    y='avg_revenue',
    color='production_company',
    text='production_company',
    labels={
        'count': 'Number of Movies',
        'avg_revenue': 'Average Revenue'
    },
    title='Production Companies: Number of Movies vs Average Revenue',
    template='plotly_white'
)

# Add labels
fig.update_traces(textposition='top center')

# Show plot
fig.show()

In [107]:
df_plots = pd.read_csv("../data/processed/df_plots.csv")

In [110]:
df_plots['release_date'].max()

'2016-09-09'

In [118]:
df_year = df_plots[df_plots['year'] == 1990]

# Encontrar la película con el máximo revenue en ese año
max_revenue_movie = df_year.loc[df_year['revenue'].idxmax()]

# Obtener el título y el revenue máximo
title = max_revenue_movie['title']
title

'Home Alone'

In [180]:
df.columns

Index(['budget', 'genres', 'original_language', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'title', 'vote_average', 'vote_count', 'director',
       'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music',
       'Mystery', 'Romance', 'Science_Fiction', 'Thriller', 'War', 'Western',
       'Other_language', 'en_language', 'es_language', 'fr_language',
       'ja_language', 'zh_language', 'Columbia Pictures', 'Dune Entertainment',
       'New Line Cinema', 'Other_company', 'Paramount Pictures',
       'Relativity Media', 'Twentieth Century Fox Film Corporation',
       'Universal Pictures', 'Village Roadshow Pictures',
       'Walt Disney Pictures', 'Warner Bros.', 'Canada', 'France', 'Germany',
       'Other_country', 'United Kingdom', 'United States of America', 'year',
       'month', 'day', 'weekday', 'day_of_year', 'gdp', 